_This notebook was put together by [Keneth Garcia](https://stivengarcia7113.wixsite.com/kenethgarcia). Source and license info are on [GitHub](https://github.com/KenethGarcia/ClassiPyGRB)._

# t-Distributed Stochastic Neighbor Embedding (t-SNE) Overview in Swift/BAT data

The Neil Gehrels Swift Observatory presents analysis results for the Swift/BAT Gamma-Ray Burst (GRBs) on [this website](https://swift.gsfc.nasa.gov/results/batgrbcat/) (open access).

In previous notebooks, we pre-process Swift Data to apply T-SNE and other dimensionality reduction techniques. In this notebook, we use the t-SNE technique to classify these pre-processed data. Through this document, we are using the _python3_ implementations from the _ClassiPyGRB_ package. It is necessary to have a _Jupyter Notebook_/_Python 3_ compiler software.

First, we need to import the _SWIFT_ instance of _ClassiPyGRB_ to our notebook (and some other packages needed):

In [1]:
from ClassiPyGRB import SWIFT
# Packages needed for this notebook
import os
import numpy as np

At this point, we need to load the data from the _SWIFT_ instance of _ClassiPyGRB_:

In [2]:
swift = SWIFT(root_path=r'type-your-path-here', res=64, n_bands=[1, 2, 3, 4, 5])
print(f"Swift/BAT bands selected: {swift.bands_selected}")

Swift/BAT bands selected: ['Time(s)', '15-25keV', '25-50keV', '50-100keV', '100-350keV', '15-350keV']


Now, from the [Preprocessing notebook](https://github.com/KenethGarcia/ClassiPyGRB/blob/20d417a5fa93f2a48ec1e5cb913484e87b05c339/docs/3.BAT_Preprocess.ipynb), extract the GRB Names, pre-processed data and index the GRB duration $T_{90}$:

In [3]:
data_loaded = np.load(os.path.join(swift.results_path, f'SWIFT_data_{swift.res}res_{"".join(str(b) for b in swift.n_bands)}bands_DFT_True.npz'))
grb_names, features = data_loaded['names'], data_loaded['data']
durations = swift.total_durations(grb_names, t=90)

Finding Durations: 100%|██████████| 1453/1453 [00:00<00:00, 2180.00GRB/s]


If you have not pre-process data yet, check the [Preprocessing notebook](https://github.com/KenethGarcia/ClassiPyGRB/blob/20d417a5fa93f2a48ec1e5cb913484e87b05c339/docs/3.BAT_Preprocess.ipynb).

# t-SNE in Swift Data

t-Distributed Stochastic Neighbor Embedding (or t-SNE) is a popular non-linear dimensionality reduction technique used for visualizing high dimensional data sets. After pre-processing Swift data in the $x_i$ vectors with Fourier Amplitudes, we want to perform this method by taking so much care when we read the results. Why? The t-SNE algorithm doesn’t always produce similar output on successive runs, and it depends on some hyperparameters related to the optimization process.

In this study, the most relevant hyperparameters on the cost function are (following the scikit-Learn and open-TSNE packages documentation):
* __Perplexity__: The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Note that perplexity linearly impacts runtime i.e. higher values of perplexity will incur longer execution time.
* __learning_rate__: The learning rate controls the step size of the gradient updates. If the learning rate is too high, the data may look like a ‘ball’ with any point approximately equidistant from its nearest neighbours. If the learning rate is too low, most points may look compressed in a dense cloud with few outliers.
* __metric__: The metric to use when calculating distance between instances in a feature array.
* __early exaggeration factor__: This factor increases the attractive forces between points and allows points to move around more freely, finding their nearest neighbors more easily.

The `perform_tsne` function of **ClassiPyGRB** only receives the pre-processed data. As optional arguments, you can pass the package used to perform t-SNE (library arg, default is 'sklearn', you can change to 'openTSNE'), and the additional kwargs for the `TSNE` instance, either of scikit-Learn or open-TSNE. The `perform_tsne` function returns the t-SNE embedding data in the 2D space.

Let's try a simple example, set the standard _perplexity_ value (30) from [Jespersen et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJ...896L..20J/abstract), set auto _learning rate_ in scikit-Learn t-SNE implementation, and perform t-SNE:

In [4]:
positions = swift.perform_tsne(features, perplexity=30, learning_rate='auto')
print(positions)

[[ -1.8333757   2.0079346]
 [ -9.940475   10.5641365]
 [-23.093807    9.295485 ]
 ...
 [ 26.193779    2.4033813]
 [ 12.170959    3.2610037]
 [ 18.019793    4.184678 ]]


As you can see, it returns a _numpy.ndarray_ with the t-SNE embedding data in the 2D space. You can use these info to plot the t-SNE embedding, for example. The same embedding using the open-TSNE package looks like this:

**Note: The scikit-learn implementation uses a different definition of `learning_rate`. In particular, openTSNE uses a definition of `learning_rate` that is 4 times smaller than scikit-Learn.**

In [5]:
positions = swift.perform_tsne(features, perplexity=30, learning_rate='auto', library='openTSNE')
print(positions)

[[ -1.22426292  -1.14724112]
 [-10.08595509  -8.00874454]
 [-21.83315516  -5.19407547]
 ...
 [ 24.02703874  -5.2522203 ]
 [ 11.1413952   -3.92316718]
 [ 16.3238586   -5.47174731]]


## t-SNE convergence

Firstly, we want to see how t-SNE converges in the pre-processed data. To do this, we use the `convergence_animation` function, it is based on the [tsne_animate](https://github.com/sophronesis/tsne_animate) implementation.

This function receives the number of frames per second to be generated, the pre-processed data, and if you want to save the animation as a _gif_ file, you can set the filename parameter to the name of the file you want to save. The default value is `None`, so the animation will not be saved. Also, you can pass any arguments of the `TSNE` instance of [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE) and the `plot_tsne` function of **ClassiPyGRB**.

Now, let's try a simple example, set the standard _perplexity_ value (30) from [Jespersen et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJ...896L..20J/abstract), set auto _learning rate_ in scikit-Learn t-SNE implementation, and perform the animation:

In [ ]:
file_name = r'/home/keneth/Documents/ClassiPyGRB/docs/files/animation1.gif'  # Change the path/filename here
gif = swift.convergence_animation(features, filename=file_name, perplexity=30, durations=durations)

![](Animations/animation1.gif)

As you can see, there is a clear dependence on $T_{90}$ duration and GRB position in the final plot (except for some GRBs, i. e. GRB190718A). Additionally, we can see that after iteration 250, the scatter pattern converges so fast. It is because (after this iteration) the TSNE instance in _scikit Learn_ updates the Kullback–Leibler divergence and `early_exaggeration` parameter.

To do more complex analysis, we can highlight custom GRBs, see redshift dependence in marker size (however, there isn't much redshift info in Swift data), and configure the TSNE running instance. For example, you can see how the GRB 211211A and GRB 060614 converges in the t-SNE instance in real-time:

In [ ]:
file_name = r'/home/keneth/Documents/ClassiPyGRB/docs/files/animation2.gif'  # Change the path/filename here
swift.convergence_animation(features, filename=file_name, perplexity=30, names=grb_names, special_cases=['GRB211211A', 'GRB060614'], special_marker_color='magenta', non_special_marker_color='silver', special_marker_size=100, marker_size=10, legend_special_cases=False)

![](Animations/animation2.gif)

## tSNE Hyperparameter review

As pioneered by [Wattenberg et al.(2016)](https://distill.pub/2016/misread-tsne/), tSNE results cannot be understood only by seeing one scatter plot in 2D. As they said: "_Getting the most from t-SNE may mean analyzing multiple plots with different perplexities._"

For this job, you can use the `tsne_animation` instance to iterate over any hyperparameter in sklearn or openTSNE. This function receives as arguments the pre-processed data, the name of the hyperparameter to iterate over (`iterable`), the frames per second of the animation (`fps`), the filename to save the animation (`filename`), and a dictionary with the additional parameters, including the iterable, `TSNE` instance parameters and `plot_tsne` arguments (`kwargs`).

## Perplexity

For example, setting default values in sklearn tSNE and iterating over **perplexity** $\in$ $[4, 500]$:

In [ ]:
pp = np.array([4, 5, 6, 7, 8, 9, 10, 15, 17, 20, 25, 30, 50, 75, 100, 150, 200, 250, 300, 350, 400, 450])
file_name = r'/home/keneth/Documents/ClassiPyGRB/docs/files/animation3.gif'  # Change the path/filename here
swift.tsne_animation(features, iterable='perplexity', perplexity=pp, filename=file_name, durations=durations, fps=1)

![](Animations/animation3.gif)

It is possible to see "pinched" shapes in the middle plot region for some TSNE initializations. Following [Wattenberg et al. 2016](https://distill.pub/2016/misread-tsne/) analysis: _"chances are the process was stopped too early"_ or this may be because the t-SNE algorithm gets stuck in a bad local minimum.

In general, lower perplexities focus on the substructure of data, and higher perplexities plots are less sensitive to small structures. By contrast, the plot structure does not change globally with perplexity (except for pinched runs), but the local structure does.

On of the reasons why high perplexity values converge better is that noisier datasets (as Swift) will require larger perplexity values to encompass enough local neighbors to see beyond the background noise (see [optimizing tSNE sklearn section](https://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne)).

However, at low perplexities (below 20), there are two or three separated groups of GRBs with significantly different duration $T_{90}$ values. This is because tSNE searches only ten to twenty neighbors for each GRB and can distinguish between these groups. Then we can use perplexity < 10 as a value to separate groups in classification.

## Learning rate

Now, we can see what happens if **learning_rate** changes within $10$ and $1000$ (values recommended in [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE)):

In [ ]:
lr = np.arange(10, 1000, 75)
file_name = r'/home/keneth/Documents/ClassiPyGRB/docs/files/animation4.gif'  # Change the path/filename here
swift.tsne_animation(features, fps=2, durations=durations, filename=file_name, iterable='learning_rate', perplexity=7, learning_rate=lr)

![](Animations/animation4.gif)

In our experience, adjusting learning rate plays a significant role in separating two well-defined subgroups at low perplexities! Meanwhile, at high perplexities learning rate only groups the data points a little.

As you can see, the learning rate doesn't change the main results globally (except if the embedding gets stuck in a bad local minimum). Instead, it can refine our results! Fine-tuning the learning rate could be critical if we need high convergence precision or well-defined map visualization.

## Early exaggeration factor

Last but not least, we have the early exaggeration factor. This parameter controls how tight natural clusters in the original space are in the embedded space and how much space will be between them. It is used only in the early exaggeration phase (the first 250 iterations in scikit Learn) and is considered not very critical. However, it recently has gotten attention in order to optimize map visualization (i.e. [here](https://www.nature.com/articles/s41467-019-13055-y)).

Let's see how this factor affects visualization maps in Swift pre-processed data. For perplexity = 5:

In [ ]:
ee = np.array([1, 2, 4, 6, 8, 10, 12, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200])
file_name = r'/home/keneth/Documents/ClassiPyGRB/docs/files/animation5.gif'  # Change the path/filename here
swift.tsne_animation(features, iterable='early_exaggeration', early_exaggeration=ee, perplexity=5, durations=durations, filename=file_name, fps=3)

![](Animations/animation5.gif)

Generally speaking, the early exaggeration factor changes the structure of the map at very low or very high values. However, it is not very critical in the middle range. It may be used as another refining parameter, similar to the learning rate but on a higher scale.